# Data Exploration & Alpha Factor Demo

Two sections:
1. **Database sanity check** — verify `stock_data.db` is built correctly and preview sample data.
2. **Alpha computation** — compute Alpha#6, #12, #38, #41, #101 via `Alpha101`.

> **Prerequisites**: run `engine.download_data()` first so that `data/stock_data.db` exists.

In [1]:
import sys
sys.path.insert(0, '../src')

import numpy as np
import pandas as pd
from data_loader import DataEngine
from alphas import Alpha101

In [2]:
engine = DataEngine()
data = engine.load_data()

df_price    = data['df_price']
df_mv       = data['df_mv']
df_industry = data['df_industry']

print('Tables loaded.')
print(f'  daily_price : {df_price.shape}  (rows = date×code combinations)')
print(f'  df_mv       : {df_mv.shape}')
print(f'  stock_info  : {df_industry.shape}')

Tables loaded.
  daily_price : (81295, 5)  (rows = date×code combinations)
  df_mv       : (81295, 1)
  stock_info  : (300, 2)


---
## Part 1 — Database Sanity Check

### 1.1 Daily Price (OHLCV)

In [3]:
dates = df_price.index.get_level_values('date')
codes = df_price.index.get_level_values('code')
print(f'Date range   : {dates.min()}  ->  {dates.max()}')
print(f'Unique stocks: {codes.nunique()}')
df_price.head(10)

Date range   : 20240102  ->  20250221
Unique stocks: 299


open   high    low  close         vol
date     code                                             
20240102 000001.SZ   9.39   9.42   9.21   9.21  1158366.45
         000002.SZ  10.44  10.48  10.15  10.15   811106.29
         000063.SZ  26.50  26.51  25.58  25.60  1124613.18
         000100.SZ   4.28   4.30   4.21   4.27  2636152.50
         000157.SZ   6.55   6.67   6.49   6.63   369054.02
         000166.SZ   4.44   4.45   4.37   4.37   566732.62
         000301.SZ   9.55   9.65   9.45   9.56   192553.78
         000333.SZ  54.63  55.05  54.20  54.56   239056.97
         000338.SZ  13.67  14.05  13.61  13.88   497134.40
         000408.SZ  25.40  26.26  25.36  25.95   111852.50

### 1.2 Market Cap (total_mv)

In [4]:
df_mv.head(10)

total_mv
date     code                   
20240102 000001.SZ  1.787285e+07
         000002.SZ  1.210967e+07
         000063.SZ  1.224512e+07
         000100.SZ  8.018667e+06
         000157.SZ  5.753509e+06
         000166.SZ  1.094246e+07
         000301.SZ  6.320320e+06
         000333.SZ  3.832620e+07
         000338.SZ  1.211246e+07
         000408.SZ  4.101229e+06

### 1.3 Industry Distribution

In [5]:
df_industry.head(10)

,name,industry
code,,
000001.SZ,平安银行,银行
000002.SZ,万科Ａ,全国地产
000063.SZ,中兴通讯,通信设备
000100.SZ,TCL科技,元器件
000157.SZ,中联重科,工程机械
000301.SZ,东方盛虹,化纤
000408.SZ,藏格矿业,农药化肥
000425.SZ,徐工机械,工程机械
000538.SZ,云南白药,中成药


In [6]:
df_industry['industry'].value_counts()

industry
银行      24
证券      22
半导体     19
电气设备    16
元器件     16
        ..
广告包装     1
乳制品      1
医药商业     1
供气供热     1
服饰       1
Name: count, Length: 66, dtype: int64

### 1.4 Missing Data Check

In [7]:
print('=== daily_price null counts ===')
print(df_price.isnull().sum())
print()
print('=== df_mv null counts ===')
print(df_mv.isnull().sum())

=== daily_price null counts ===
open     0
high     0
low      0
close    0
vol      0
dtype: int64

=== df_mv null counts ===
total_mv    0
dtype: int64


---
## Part 2 — Alpha Factor Computation

Using `Alpha101` from `src/alphas.py`, which implements alphas from  
*'101 Formulaic Alphas'* (Kakushadze, 2015).  
Raw values (NaN, inf) are preserved — data cleaning comes later.

In [8]:
alpha = Alpha101(data)
print('Alpha101 initialized.')
print(f'  Price matrix shape (dates × codes): {alpha.close.shape}')

Alpha101 initialized.
  Price matrix shape (dates × codes): (273, 299)


/Users/infinity/Desktop/Career/Quant/Multi-factor Model for Stock Selection/Multi-factor-Model-for-Stock-Selection/notebooks/../src/alphas.py:57: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  self.returns  = self.close.pct_change()


### 2.1 Individual Alphas (wide form: dates × codes)

In [9]:
# Alpha#6: -1 * correlation(open, volume, 10)
a6 = alpha.alpha006()
print('Alpha#6  shape:', a6.shape)
a6.tail(5).iloc[:, :6]   # last 5 dates, first 6 stocks

Alpha#6  shape: (273, 299)


code,000001.SZ,000002.SZ,000063.SZ,000100.SZ,000157.SZ,000166.SZ
date,,,,,,
20250217,-0.524517,-0.248834,-0.427437,-0.025062,0.254170,-0.000582
20250218,-0.725675,-0.188596,-0.542344,-0.008392,0.142601,0.051444
20250219,-0.556824,-0.187760,-0.606402,0.194334,0.196593,0.164627
20250220,-0.311493,-0.044974,-0.595116,0.024773,0.197812,0.038002
20250221,-0.309479,-0.213198,-0.146451,0.122155,-0.057652,-0.505223


In [10]:
# Alpha#12: sign(delta(volume, 1)) * (-1 * delta(close, 1))
a12 = alpha.alpha012()
print('Alpha#12 shape:', a12.shape)
a12.tail(5).iloc[:, :6]

Alpha#12 shape: (273, 299)


code,000001.SZ,000002.SZ,000063.SZ,000100.SZ,000157.SZ,000166.SZ
date,,,,,,
20250217,-0.23,-0.13,-1.16,-0.02,0.05,-0.06
20250218,0.03,-0.16,-1.61,0.11,-0.13,-0.11
20250219,-0.10,0.13,-1.10,-0.04,-0.21,0.05
20250220,-0.05,-0.13,-0.57,0.01,0.05,-0.04
20250221,0.02,-0.10,-3.59,-0.02,0.05,-0.09


In [11]:
# Alpha#38: (-1 * rank(ts_rank(close, 10))) * rank(close / open)
a38 = alpha.alpha038()
print('Alpha#38 shape:', a38.shape)
a38.tail(5).iloc[:, :6]

Alpha#38 shape: (273, 299)


code,000001.SZ,000002.SZ,000063.SZ,000100.SZ,000157.SZ,000166.SZ
date,,,,,,
20250217,-0.751033,-0.142421,-0.240842,-0.220225,-0.479535,-0.314662
20250218,-0.754439,-0.230242,-0.022626,-0.027474,-0.223244,-0.122960
20250219,-0.095172,-0.451168,-0.403038,-0.014725,-0.826648,-0.438632
20250220,-0.281812,-0.135064,-0.121613,-0.206168,-0.760039,-0.148111
20250221,-0.142566,-0.273095,-0.866864,-0.124831,-0.142280,-0.502724


In [12]:
# Alpha#41: sqrt(high * low) - vwap
a41 = alpha.alpha041()
print('Alpha#41 shape:', a41.shape)
a41.tail(5).iloc[:, :6]

Alpha#41 shape: (273, 299)


code,000001.SZ,000002.SZ,000063.SZ,000100.SZ,000157.SZ,000166.SZ
date,,,,,,
20250217,-0.035669,0.020658,-0.081462,-0.001970,-0.009412,0.011265
20250218,0.016245,0.030286,0.180518,0.044714,0.027570,0.014591
20250219,0.011487,-0.017614,-0.182166,0.004421,-0.039413,-0.010243
20250220,0.014871,0.008133,0.017347,0.001355,-0.016912,0.003246
20250221,-0.003608,-0.004280,-0.578808,-0.005579,0.015991,-0.017141


In [13]:
# Alpha#101: (close - open) / (high - low + 0.001)
a101 = alpha.alpha101()
print('Alpha#101 shape:', a101.shape)
a101.tail(5).iloc[:, :6]

Alpha#101 shape: (273, 299)


code,000001.SZ,000002.SZ,000063.SZ,000100.SZ,000157.SZ,000166.SZ
date,,,,,,
20250217,0.717131,-0.278884,0.482426,0.360360,0.079681,-0.305344
20250218,0.248756,-0.412371,-0.735189,-0.391459,-0.616114,-0.763359
20250219,-0.687023,0.622407,0.693610,-0.132450,0.876494,0.594059
20250220,-0.450450,-0.630631,-0.401737,0.180180,0.495868,-0.491803
20250221,-0.310559,0.373444,0.763032,0.000000,-0.348259,0.567376


### 2.2 Combined Alpha DataFrame  (MultiIndex: date × code)

In [14]:
df_alphas = alpha.get_all_alphas()
print('Combined alpha DataFrame shape:', df_alphas.shape)
print('Columns:', df_alphas.columns.tolist())
df_alphas.head(10)

Combined alpha DataFrame shape: (81627, 5)
Columns: ['alpha006', 'alpha012', 'alpha038', 'alpha041', 'alpha101']


/Users/infinity/Desktop/Career/Quant/Multi-factor Model for Stock Selection/Multi-factor-Model-for-Stock-Selection/notebooks/../src/alphas.py:245: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  stacked = wide.stack(dropna=False)   # Series: MultiIndex (date, code)
/Users/infinity/Desktop/Career/Quant/Multi-factor Model for Stock Selection/Multi-factor-Model-for-Stock-Selection/notebooks/../src/alphas.py:245: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  stacked = wide.stack(dropna=False)   # Series: MultiIndex (date, code)
/Users/infinity/Desktop/Career/Quant/Multi-facto

alpha006  alpha012  alpha038  alpha041  alpha101
date     code                                                       
20240102 000001.SZ       NaN       NaN       NaN  0.034408 -0.853081
         000002.SZ       NaN       NaN       NaN  0.053680 -0.876133
         000063.SZ       NaN       NaN       NaN  0.144182 -0.966702
         000100.SZ       NaN       NaN       NaN -0.005238 -0.109890
         000157.SZ       NaN       NaN       NaN -0.017282  0.441989
         000166.SZ       NaN       NaN       NaN  0.013152 -0.864198
         000301.SZ       NaN       NaN       NaN -0.003857  0.049751
         000333.SZ       NaN       NaN       NaN  0.020013 -0.082256
         000338.SZ       NaN       NaN       NaN -0.018417  0.476190
         000408.SZ       NaN       NaN       NaN -0.050590  0.610433

### 2.3 Descriptive Statistics

In [15]:
df_alphas.describe()

,alpha006,alpha012,alpha038,alpha041,alpha101
count,78505.000000,80985.000000,78505.000000,81295.000000,81295.000000
mean,-0.128252,-0.242750,-0.291922,-0.002396,-0.002045
std,0.431199,2.772705,0.263080,0.454496,0.525875
min,-0.994983,-130.000000,-0.993289,-23.553738,-0.999947
25%,-0.467123,-0.340000,-0.470025,-0.036370,-0.455595
50%,-0.142080,-0.040000,-0.211389,0.001541,0.000000
75%,0.195562,0.120000,-0.064321,0.045615,0.447651
max,0.975150,101.960000,-0.000011,20.348329,0.999976


### 2.4 NaN Coverage per Alpha

In [16]:
total = len(df_alphas)
null_pct = df_alphas.isnull().sum() / total * 100
null_pct.rename('NaN %').to_frame()

,NaN %
alpha006,3.824715
alpha012,0.786504
alpha038,3.824715
alpha041,0.406728
alpha101,0.406728


### 2.5 Cross-Sectional Snapshot on the Latest Date

In [17]:
latest_date = df_alphas.index.get_level_values('date').max()
print(f'Latest date: {latest_date}')
df_alphas.loc[latest_date].dropna().head(10)

Latest date: 20250221


,alpha006,alpha012,alpha038,alpha041,alpha101
code,,,,,
000001.SZ,-0.309479,0.02,-0.142566,-0.003608,-0.310559
000002.SZ,-0.213198,-0.10,-0.273095,-0.004280,0.373444
000063.SZ,-0.146451,-3.59,-0.866864,-0.578808,0.763032
000100.SZ,0.122155,-0.02,-0.124831,-0.005579,0.000000
000157.SZ,-0.057652,0.05,-0.142280,0.015991,-0.348259
000166.SZ,-0.505223,-0.09,-0.502724,-0.017141,0.567376
000301.SZ,-0.102671,0.17,-0.032774,0.032392,-0.613027
000333.SZ,0.454647,0.64,-0.024865,0.039791,-0.417071
000338.SZ,-0.208885,-0.96,-0.840237,-0.102937,0.828604
